In [1]:
import pandas as pd
import os
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
from sklearn.cluster import HDBSCAN, KMeans
import time
from tqdm import tqdm
from sklearn.preprocessing import normalize
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import calinski_harabasz_score,silhouette_score
from sklearn.metrics.pairwise import cosine_similarity,euclidean_distances
from sklearn.decomposition import PCA
from umap import UMAP
import optuna
from algorithms import *

/home/hevagog/Studies/advanced-data-mining/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
questions = np.load(os.path.join(Path.cwd().parent.parent,'data', "question_embeddings.npy"))
tags = pd.read_parquet(os.path.join(Path.cwd().parent.parent,'data', "tag_embeddings.parquet"))

In [3]:
tags.embedding.values
t_stacked = np.stack(tags.embedding.values)
t_stacked.shape

(22753, 4096)

In [ ]:
# np.linalg.norm(t_stacked, axis=1, keepdims=True)

In [ ]:
questions[:10_000].shape

(10000, 4096)

In [ ]:
# np.linalg.norm(questions[:10_000], axis=1, keepdims=True)


In [ ]:
matcher = DualEncoderMatcher(questions[:10_000], t_stacked, tags.index)

In [ ]:
predictions = matcher.predict_top_k(question_idx=0, k=5)
print(f"Top 5 predictions: {predictions}")


Top 5 predictions: [('entity-framework', np.float64(0.5814004526467879)), ('linq-to-entities', np.float64(0.5646984342057525)), ('entity-framework-6', np.float64(0.5354078775941149)), ('entity-framework-5', np.float64(0.5260456250104916)), ('entity-framework-4.1', np.float64(0.5250464008985329))]


In [ ]:

batch_predictions = matcher.batch_predict(
    question_indices=range(1000),  # First 1000 questions
    k=5
)

In [ ]:
batch_predictions[0]

[('entity-framework', np.float64(0.5814004526467882)),
 ('linq-to-entities', np.float64(0.5646984342057526)),
 ('entity-framework-6', np.float64(0.535407877594115)),
 ('entity-framework-5', np.float64(0.5260456250104918)),
 ('entity-framework-4.1', np.float64(0.5250464008985329))]

In [ ]:
original_data = pd.read_csv(os.path.join(Path.cwd().parent,'data', 'stackexchange_dataset.csv'))

In [ ]:
original_data.iloc[0]['tags']

"['c#', 'entity-framework']"

In [ ]:
original_data.iloc[0]['question_text']

'I am looking for a better way to use the .Include clause of Entity Framework. I want to avoid duplicate code. I have a lot of classes, and every class has a method which looks something like this: query = (from n in currentDBContext.FBBuchungenCollection .Include(x => x.BelegHerkunft) .Include(x => x.Buchungsordner) .Include(x => x.Buchungsperiode).ThenInclude(x => x.Geschaeftsjahr) .Include(x => x.BuchungsUser) .Include(x => x.Erfassungsart) .Include(x => x.ErstellUser) .Include(x => x.Mandant).ThenInclude(x => x.HauptAdresse) .Include(x => x.StornoUser) .Include(x => x.Teilbuchungen).ThenInclude(x => x.FremdWaehrung) .Include(x => x.Teilbuchungen).ThenInclude(x => x.KKArt) .Include(x => x.Teilbuchungen).ThenInclude(x => x.KKKonto) .Include(x => x.Teilbuchungen).ThenInclude(x => x.Konto) .Include(x => x.Teilbuchungen).ThenInclude(x => x.KoReVerteilung).ThenInclude(x => x.Periodenverteilungen).ThenInclude(x => x.Kontierungen).ThenInclude(x => x.Kontierungsangaben).ThenInclude(x => x.K

In [4]:
tree_builder = DirectHierarchicalLabelTree(
    tag_embeddings=t_stacked,
    tag_names=tags.index,
    branching_factors=[100, 100],
    use_spherical_clusters=True
)

In [5]:
tree_builder.build_tree()

{'root': {'is_leaf': False,
  'n_clusters': 100,
  'centroids': array([[ 6.0751815e-03, -2.3715999e-03, -2.0886557e-02, ...,
          -4.0854649e-03, -1.6738884e-02,  7.3872884e-03],
         [ 3.3007376e-02,  7.0581846e-03, -4.1581751e-03, ...,
          -1.2421027e-02, -6.0356050e-03, -1.4805568e-03],
         [ 1.0641773e-02, -1.9808197e-02, -1.0966040e-02, ...,
          -2.8894788e-03, -1.6938521e-02,  1.6713377e-02],
         ...,
         [ 2.5996938e-02,  2.8620737e-03,  2.6476844e-03, ...,
          -2.9850837e-03,  4.2769923e-05,  5.1233750e-03],
         [ 1.7368624e-02, -1.2699831e-02, -3.4959968e-03, ...,
          -1.2151112e-02, -5.4544122e-03,  1.5990483e-02],
         [ 1.7617881e-02, -8.6007472e-03, -9.0531623e-03, ...,
          -4.8421426e-03, -1.8730758e-03,  1.5169734e-02]],
        shape=(100, 4096), dtype=float32),
  'children': ['root_c0',
   'root_c1',
   'root_c2',
   'root_c3',
   'root_c4',
   'root_c5',
   'root_c6',
   'root_c7',
   'root_c8',
   'root_c

In [6]:
tree_builder.visualize_tree_statistics()

Total tags: 22753
Level 0: 1 clusters
Level 1: 100 clusters
Level 2: 9752 clusters


In [7]:
h1_t = find_representative_tags(tree_builder, tags.index)

In [8]:
centroid_tags = pd.DataFrame.from_dict(h1_t, orient='index')
centroid_tags.index = centroid_tags['closest_tag'].values
centroid_tags.drop(columns=['closest_tag'], inplace=True)
centroid_tags.head(5)

,closest_dist,cluster_mean,tag_embedding,cluster_size,top_5_popular_tags
facebook,0.427348,"[0.004879954, -0.0019050128, -0.016777342, -0....","[0.002688237, 0.015740797, -0.03034449, -0.018...",94,"[instagram-api, instagram, instagram-graph-api..."
decimal,0.241076,"[0.027635492, 0.0065359636, -0.0029253308, -0....","[0.03159611, 0.00029558412, -0.006964058, -0.0...",370,"[intervals, differential-equations, math, maxi..."
storage,0.417411,"[0.008282, -0.016636355, -0.0092419945, -0.004...","[0.020607475, -0.017888447, -0.00067037623, -0...",134,"[dicomweb, remotestorage, storage, multipartfo..."
tf-idf,0.309486,"[0.027263727, -0.007913517, 0.0023698097, -0.0...","[0.024365697, -0.010216675, 0.0055818437, -0.0...",243,"[wpml, locale, dictionary, spacy, microsoft-tr..."
qt,0.397460,"[0.01614129, 0.005696066, 0.016760174, -0.0084...","[0.016203284, 0.013681211, 0.018016785, -0.007...",106,"[qt, qml, pyqt, qasync, qt6]"


In [10]:
h1_t = find_representative_tags(tree_builder, tags.index,similarity_metric='cosine')

In [11]:
centroid_tags = pd.DataFrame.from_dict(h1_t, orient='index')
centroid_tags.index = centroid_tags['closest_tag'].values
centroid_tags.drop(columns=['closest_tag'], inplace=True)
centroid_tags.head(5)

,closest_dist,cluster_mean,tag_embedding,cluster_size,top_5_popular_tags
facebook,0.089584,"[0.004879954, -0.0019050128, -0.016777342, -0....","[0.002688237, 0.015740797, -0.03034449, -0.018...",94,"[instagram-api, instagram, instagram-graph-api..."
integer,0.040206,"[0.027635492, 0.0065359636, -0.0029253308, -0....","[0.03159611, 0.00029558412, -0.006964058, -0.0...",370,"[intervals, differential-equations, math, maxi..."
storage,0.105308,"[0.008282, -0.016636355, -0.0092419945, -0.004...","[0.020607475, -0.017888447, -0.00067037623, -0...",134,"[dicomweb, remotestorage, storage, multipartfo..."
tf-idf,0.070302,"[0.027263727, -0.007913517, 0.0023698097, -0.0...","[0.024365697, -0.010216675, 0.0055818437, -0.0...",243,"[wpml, locale, dictionary, spacy, microsoft-tr..."
qt,0.074746,"[0.01614129, 0.005696066, 0.016760174, -0.0084...","[0.016203284, 0.013681211, 0.018016785, -0.007...",106,"[qt, qml, pyqt, qasync, qt6]"


In [18]:
centroid_tags.where(centroid_tags['top_5_popular_tags'].apply(lambda x: 'python' in x)).dropna()

,closest_dist,cluster_mean,tag_embedding,cluster_size,top_5_popular_tags
pymacs,0.027905,"[0.0025966496, 0.0051004626, -0.014530499, -0....","[0.0038225949, 0.0024991601, -0.009915985, -0....",315.0,"[python, jupyter-notebook, jupyter, python-log..."


In [12]:
matcher = DualEncoderMatcher(
    question_embeddings=questions,
    tag_embeddings=np.vstack(centroid_tags['cluster_mean'].values),
    tag_names=centroid_tags.index.tolist(),
)

In [14]:
top_k = matcher.predict_top_k(1, k=5)
top_k

[('window', np.float64(0.38430531693978565)),
 ('datetime2', np.float64(0.38051967184035895)),
 ('tcpstream', np.float64(0.361597034135499)),
 ('archlinux', np.float64(0.3328057418352225)),
 ('interprocess', np.float64(0.3292823141840308))]

In [15]:
centroid_tags.to_parquet(os.path.join(Path.cwd().parent,'data', 'centroid_spherical_tags.parquet'))

In [17]:
centroid_tags.head(5)

,closest_dist,cluster_mean,tag_embedding,cluster_size,top_5_popular_tags
facebook,0.089584,"[0.004879954, -0.0019050128, -0.016777342, -0....","[0.002688237, 0.015740797, -0.03034449, -0.018...",94,"[instagram-api, instagram, instagram-graph-api..."
integer,0.040206,"[0.027635492, 0.0065359636, -0.0029253308, -0....","[0.03159611, 0.00029558412, -0.006964058, -0.0...",370,"[intervals, differential-equations, math, maxi..."
storage,0.105308,"[0.008282, -0.016636355, -0.0092419945, -0.004...","[0.020607475, -0.017888447, -0.00067037623, -0...",134,"[dicomweb, remotestorage, storage, multipartfo..."
tf-idf,0.070302,"[0.027263727, -0.007913517, 0.0023698097, -0.0...","[0.024365697, -0.010216675, 0.0055818437, -0.0...",243,"[wpml, locale, dictionary, spacy, microsoft-tr..."
qt,0.074746,"[0.01614129, 0.005696066, 0.016760174, -0.0084...","[0.016203284, 0.013681211, 0.018016785, -0.007...",106,"[qt, qml, pyqt, qasync, qt6]"


---

In [ ]:
tree_builder.tag_to_path[0]

'root_c16_c12'

In [ ]:
tree_builder.tag_to_path[0].split('_c')

['root', '16', '12']

In [ ]:
np.mean(tree_builder.tree['root_c16']['centroids'], axis=0)

array([ 0.00887133,  0.00042595, -0.00304763, ..., -0.00895712,
       -0.00428712,  0.0040433 ], shape=(4096,))

In [ ]:
tst_mean = np.mean(tree_builder.tree['root_c16']['centroids'], axis=0)
tst_v = np.stack(tree_builder.tree[tree_builder.tag_to_path[1]]['centroid'])

In [ ]:
euclidean_distances(tst_mean.reshape(1, -1), tst_v.reshape(1, -1))[0][0]

np.float64(0.5633195376844613)

In [ ]:
children_0_tags=[]
closest_tag,closest_dist = None, float('inf')
for tag_name,i in zip(tags.index,range(tags.index.shape[0])):
    tag_path = tree_builder.tag_to_path[i]
    tag_path_parts = tag_path.split('_c')
    if tag_path_parts[1] == '0':
        embedding_cluster_mean = np.mean(tree_builder.tree[tag_path_parts[0] + '_c' + tag_path_parts[1]]['centroids'], axis=0)
        children_0_tags.append(tag_name)
        embedding_val = np.stack(tree_builder.tree[tag_path]['centroid'])
        dist = euclidean_distances(embedding_val.reshape(1, -1), embedding_cluster_mean.reshape(1, -1))[0][0]
        if dist < closest_dist:
            closest_dist = dist
            closest_tag = tag_name
    
    

In [ ]:
h1_t = {centroid_idx: [] for centroid_idx in range(100)}


In [ ]:
len(tree_builder.tree['root']['children'])

100

In [ ]:
h1_t = find_representative_tags(tree_builder, tags, n_centroids=100)

In [ ]:
centroid_100_tags = pd.DataFrame.from_dict(h1_t, orient='index')
centroid_100_tags.index = centroid_100_tags['closest_tag'].values
centroid_100_tags.drop(columns=['closest_tag'], inplace=True)
centroid_100_tags

,closest_dist,cluster_mean,tag_embedding
denodb,0.320131,"[0.014417376148063496, -0.007384050656546192, ...","[0.0116502746, -0.008069663799999998, -0.01297..."
memory,0.385309,"[0.028688511345533328, 0.0014271237903345237, ...","[0.034116245, 0.010937103414285715, -0.0185003..."
database-administration,0.322269,"[0.007003796571137018, -0.01554148007059889, 0...","[0.014782062733333332, -0.012128053166666665, ..."
rpc,0.256773,"[0.013661504061644281, -0.002496577893879999, ...","[0.0054010459, -0.0003999848199999998, -0.0013..."
stablexui,0.352327,"[0.025036833951179758, -0.00036814772256623003...","[0.026074393999999994, 0.0023631919622222218, ..."
...,...,...,...
pytorch,0.380658,"[0.016381107836795236, 0.013594593070475718, -...","[0.008694216432857142, 0.025070296428571424, -..."
field,0.269966,"[0.017792385638042314, 0.00044468486129393747,...","[0.013452607600000001, 0.010712051466666667, -..."
netlify,0.387222,"[0.01575377066778831, -0.004307782403267663, -...","[0.009402036083333334, -0.004371085221666666, ..."
word-table,0.370249,"[0.01430713080853095, -0.010452169643353429, -...","[0.016797865071428573, -0.018058393357142856, ..."


In [ ]:
centroids_lite = centroid_100_tags.copy()
centroids_lite = centroids_lite.drop(columns=['closest_dist', 'tag_embedding'])
# centroids_lite.to_parquet(os.path.join(Path.cwd().parent,'data', 'centroid_100_tags_lite.parquet'))
centroids_lite

,cluster_mean
denodb,"[0.014417376148063496, -0.007384050656546192, ..."
memory,"[0.028688511345533328, 0.0014271237903345237, ..."
database-administration,"[0.007003796571137018, -0.01554148007059889, 0..."
rpc,"[0.013661504061644281, -0.002496577893879999, ..."
stablexui,"[0.025036833951179758, -0.00036814772256623003..."
...,...
pytorch,"[0.016381107836795236, 0.013594593070475718, -..."
field,"[0.017792385638042314, 0.00044468486129393747,..."
netlify,"[0.01575377066778831, -0.004307782403267663, -..."
word-table,"[0.01430713080853095, -0.010452169643353429, -..."


In [ ]:
dem = DualEncoderMatcher(
    question_embeddings=questions[:10_000],
    tag_embeddings=centroids_lite['cluster_mean'].values.tolist(),
    tag_names=centroids_lite.index.tolist()
)

In [ ]:
original_data.iloc[2]['question_text']

'I am new to pandas library in python. When I loaded a file and was printing the output of df.info into console, the data is getting printed first instead of the text that I have printed. What is causing this behavior(since it was not async I couldn\'t understand how it is behaving) import os import pandas as pd dataset_directory_path = "/home/user/python-temp/datasets" file_datasets = os.listdir(dataset_directory_path) datasets = dict() for file_dataset in file_datasets: dataset_name = file_dataset.split(\'.\')[0] dataset_data = pd.read_csv(os.path.join(dataset_directory_path, file_dataset)) datasets[dataset_name] = dataset_data un_processed_customers_data = datasets["customers"] print("Data structure for Df", un_processed_customers_data.info()) Ouput: <class \'pandas.core.frame.DataFrame\'> RangeIndex: 10000 entries, 0 to 9999 Data columns (total 12 columns): # Column Non-Null Count Dtype --- ------ -------------- ----- 0 Index 10000 non-null int64 1 Customer Id 10000 non-null object

In [ ]:
predictions = dem.predict_top_k(question_idx=2, k=5)
print(f"Top 5 predictions: {predictions}")


Top 5 predictions: [('python', np.float64(0.38295367797329094)), ('word-table', np.float64(0.3696944707340029)), ('ironpdf', np.float64(0.3647400168217892)), ('django', np.float64(0.3297345619706671)), ('apache-spark', np.float64(0.3283243442282066))]
